In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import cv2

# from google.colab.patches import cv2_imshow


# Load the iris dataset

In [ ]:
# load the iris dataset
df = pd.read_csv('iris.csv')
df.head()

In [ ]:
#dataframe of feature
df_feature=df.loc[:, ["sepal.length",	"sepal.width", "petal.length","petal.width"]]

#df_feature.head()

#dataframe of label
df_label=df.loc[:, ["variety"]]

#df_label.head()

# Create a dataframe of feature and a dataframe of label

# Manual Hyperparameters tuning

*  Splitting training/testing
*  Freezing the test set
*  Spitting training/validation (to retrieve a validation set for hyperparameters tuning)
*  Defining hyperparameters to be tuned (in this case 2 and 2 hyperparameters)
*  Initializing 4 classifiers with the 4 combination of hyperparameters
*  Training the 4 classifiers
*  Testing the 4 classifiers performance on the validation set
*  Choosing the best combination of hyperparameters

In [ ]:
# Train/Test split (10% of the training dataset will be used for testing)
x_train, x_test, y_train, y_test = train_test_split(df_feature, df_label, test_size = .10)

print("original set len", len(df_feature))
print("training set len", len(x_train))
print("test set len", len(x_test))

# Train/Test split (20% of the new training dataset will be used for validation)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=.20)

print("training set new len", len(x_train))
print("valid set len", len(x_valid))

Training the different classifiers (i.e., the same classifier with different combinantion of hyperparameters).
* n_estimators = number of decison trees (typically between 10 and 1000).
* max_depth = defines the maximum depth of the tree, i.e. the number of decision nodes in each decision tree. If `None`, then nodes are expanded until all leaves are pure.

We want to tune the following hyperparameters:
* n_estimators = 100,200
* max_depth = 2,3


In [ ]:
clf1 = RandomForestClassifier(n_estimators = 100, max_depth = 2)
clf2 = RandomForestClassifier(n_estimators = 100, max_depth = 3)
clf3 = RandomForestClassifier(n_estimators = 200, max_depth = 2)
clf4 = RandomForestClassifier(n_estimators = 200, max_depth = 3)

trainedclf1=clf1.fit(x_train, y_train)
trainedclf2=clf2.fit(x_train, y_train)
trainedclf3=clf3.fit(x_train, y_train)
trainedclf4=clf4.fit(x_train, y_train)

Validating the best classifier (i.e., the one with the best combination of hyperparameters (i.e., the one with the highest performance on the validation set))

In [ ]:
prediction1=trainedclf1.predict(x_valid)
prediction2=trainedclf2.predict(x_valid)
prediction3=trainedclf3.predict(x_valid)
prediction4=trainedclf4.predict(x_valid)

print(accuracy_score(y_valid, prediction1))
print(accuracy_score(y_valid, prediction2))
print(accuracy_score(y_valid, prediction3))
print(accuracy_score(y_valid, prediction4))

We finally test the performance of the best classifier after hyperparameters tuning on the test set

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

predictiontestset=trainedclf1.predict(x_test)

# Calculate the accuracy
print(accuracy_score(y_test, predictiontestset))

#confusion matrix
classes=["Setosa", "Virginica", "Versicolor"]
cm = confusion_matrix(y_test, predictiontestset, labels=classes)
cmd = ConfusionMatrixDisplay(cm, display_labels=classes)

cmd.plot()


## Hyperparameters tuning: a more complex but accepted scenario

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data.

This situation is called **overfitting**.

To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set X_test, y_test.


When evaluating different settings (“hyperparameters”) for estimators, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally.

This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance.

To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is **cross-validation**. A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets.

The following procedure is followed for each of the k “folds”:

- A model is trained using k-1 of the folds as training data;

- the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop.

In [ ]:
# configure the cross-validation procedure
loop = KFold(n_splits=3, shuffle=True, random_state=1)

# define the model
clf = RandomForestClassifier()

# define search space
grid = dict()
grid['n_estimators'] = [10, 100, 500]
grid['max_depth'] = [2, 4, 6]

# define search for hyperparameters tuning
search = GridSearchCV(clf, grid, scoring='accuracy', cv=loop)

# execute search
result = search.fit(x_train, y_train)

In [ ]:
# get the best performing model fit on the whole training dataset (i.e., the one with the best hyperparameters)
best_model = result.best_estimator_
print("the best model is ", best_model)

In [ ]:
#predictions on the test set
predictions=best_model.predict(x_test)
print("the accuracy is ", accuracy_score(y_test,predictions))

In [ ]:

# A global overview of cross validation
search.cv_results_

In [ ]:
#print keys of interest for viewing hyperparamenters.
print(search.cv_results_['params'])
print(search.cv_results_['mean_test_score']) #mean of performance of each fold to discriminate hyperparameters